In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

In [4]:
import os
import gc
import time
from self_control.suffix_gradient.utils import get_verbalized_grads, get_verbalized_grads_from_wrapped_model
os.environ["CUDA_VISIBLE_DEVICES"]="6"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from itertools import islice
import torch
from tqdm import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from self_control.suffix_gradient.repe import WrappedReadingVecModel
import torch.nn.functional as F
from peft import AdaptionPromptConfig, get_peft_model, LoraModel, LoraConfig, prepare_model_for_kbit_training

In [6]:
# model_name_or_path = "/home/models/llama2-7b-chat-hf/"
model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True).eval()
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_d|type=torch.float32, device_map="auto", token=True).eval()
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
loss_fct = torch.nn.CrossEntropyLoss()
wrapped_model = WrappedReadingVecModel(model.eval(), tokenizer)

In [8]:
DEFAULT_SYSTEM_PROMPT = """<<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> """
user_tag = "[INST]"
assistant_tag = "[/INST]"

In [5]:
gsm8k_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/gsm8k/test.jsonl", 'r') as f:
    for line in f:
        gsm8k_data.append(eval(line))

In [10]:
# gather target hiddens
from self_control.suffix_gradient.utils import SuffixItem
suffix = SuffixItem(suffix=f" {user_tag} Was your last reasoning and anwer accurate? Give answer as \"Yes, it was accurate\" or \"No, it was not accurate\". {assistant_tag} Answer: ", target="Yes")

In [11]:
wrapped_model.reset()
for i in tqdm(range(20)):
    data_item = gsm8k_data[i]
    ground_truth = data_item['answer'].split("\n#### ")[-1]
    input = f"Q: {data_item['question']}\n A: "
    hidden_states = wrapped_model.controlled_generate(
        prompt=input,
        suffix=suffix,
        loss_fct=loss_fct,
        coeff=-0.1,
        iterations=2,
        random_seed=0,
        smoothing=0,
        # norm=5,
        # verbose=True,
        return_hiddens=True,
        max_new_tokens=200,
        return_intermediate=True,
        # search=True,
        gradient_manipulation="clipping",
        annealing=0.8,
        # consistent=False,
    )
    # for iter in range(len(iterative_outputs)):
    #     control_acc_dict[iter] += eval_answer(ground_truth, iterative_outputs[iter])
    # print(controlled_answer)
    wrapped_model.reset()
    break
    # time.sleep(1)
# print(iterative_outputs)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:46<?, ?it/s]


In [14]:
from 
hidden_states

torch.Size([1, 79, 4096])